# Analysis- Basic
* 고속도로 기본구간(1번 영상) 분석

# Imports

In [1]:
import pandas as pd
import numpy as np

import os
import warnings
from tqdm import tqdm

from plotly.subplots import make_subplots
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as po

import statistics
import math

# Analysis
* `01_denstiy_by_target_LOS_concated`의 파일을 불러오기
* 필요한 분석 데이터들을 추출하여 각각의 파일로 저장하기

## Load Dataset

### 궤적 데이터

In [2]:
folder_dir = 'D:/OneDrive - 연세대학교 (Yonsei University)/Projects/Yonsei_TELab/003_도로상충_210517-/2차년도_2022/21_드론궤적분석자료_영업소_서울TG_부산방향/다차로하이패스'

In [3]:
folder_name = '01_denstiy_by_target_LOS_concated'

file_dir = os.path.join(folder_dir, folder_name)

In [4]:
file_list = os.listdir(file_dir)
file_list[0:3]

['01_1_A.csv', '01_2_A.csv', '01_3_A.csv']

In [5]:
df_list = []
num_LOS_list = []

for file in file_list:
    num_LOS = file[0:6] # 파일명으로부터 01_2_C 형식의 대상지 번호와 LOS 추출
    
    file_path = os.path.join(file_dir, file)
    
    globals()[f'target{num_LOS}'] = pd.read_csv(file_path, encoding = 'cp949')
    
    df_list.append(globals()[f'target{num_LOS}'])
    num_LOS_list.append(num_LOS)

In [6]:
df_list[0].head(3)

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Vehicle ID,Frame ID,Total Frames,Global Time (Epoch Time),Local X (m),Local Y(m),...,Vehicle Acceleration,Lane Identification,Preceding Vehicle,Following Vehicle,Spacing Headway,Time Headway,Bad Object,Special Car,Lane Class,Vehicle Movement
0,0,12,12,12,2,384,74,46812800,71.513092,23.467445,...,0.273768,U2,0,4,0.00000,0.000000,0,0,상행본선,2.478930
1,1,13,13,13,2,387,77,46812900,73.919029,23.549143,...,-7.160653,U2,0,4,0.00000,0.000000,0,0,상행본선,2.407324
2,2,14,14,14,4,387,37,46812900,43.158875,25.920057,...,-0.548999,U2,2,0,31.03002,1.158508,0,0,상행본선,2.678447


### Load tmsec info
* 총 관측시간 데이터 로드

In [7]:
tmsec_info = pd.read_csv('tmsec_A_MH.csv') # 대상지별, LOS별 동영상 지속시간(tmsec) 정보 불러오기

In [8]:
tmsec_info.head()

,Unnamed: 0,num,LOS,tmsec
0,0,01_1,A,287.0
1,1,01_2,A,218.2
2,2,01_3,A,286.0
3,3,01_4,A,287.1


## 평균 속도 정보
1. 차량(Vehicle ID)별 누적 이동거리(%) 구하기
2. 전체 길이를 일정 간격의 구간으로 나누기
3. 차선별, 일정 구간별(누적길이%) 속도 히트맵 그리기

### 교통류율, 전체 공간평균속도(SMS)
* 차선별 총 교통량
| lane | flowrate | SMS |
|------|--------|-----|
| U1   | 200    | 80  |
| U2   | 300    | 90  |
| U3   | 100    | 80  |

### 필요 함수 정의
* 공간평균속도는 조화평균(harmonic_mean)을 사용해 구함
* 조화평균함수는 `statistics.harmonic_mean()`을 이용할 수 있음

In [9]:
# 조화평균 함수
harmonic = lambda x: statistics.harmonic_mean(list(x))
harmonic .__name__ = 'harmonic'

In [10]:
# 고유값 개수 산정 함수(유니크)
unq = lambda x: len(x.unique())
unq .__name__ = 'unq'

In [11]:
def LOS_expwy_basic(x:int):
    """밀도를 바탕으로 LOS를 판정해 주는 함수 :: 고속도로 기본구간 기준"""
    
    if x >= 0 and x <= 6:
        return 'A'
    elif x > 6 and x <= 10:
        return 'B'
    elif x > 10 and x <= 14:
        return 'C'
    elif x > 14 and x <= 19:
        return 'D'
    elif x > 19 and x <= 28:
        return 'E'
    elif x > 28:
        return 'F'
    
    else:
        pass

### 전체 차량대수, 공간평균속도, 교통류율 산정 후 저장
* 각 대상지별/LOS별 전체 차량대수, 공간평균속도(SMS), 교통류율 구하여 저장하기
* `harmonic` 함수 사용 : `harmonic(target1_1_A['Vehicle Velocity(km/h)'])` <- 이런 식으로 쓸 수 있다

In [12]:
df_list[0].columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1', 'Unnamed: 0.1.1.1',
       'Vehicle ID', 'Frame ID', 'Total Frames', 'Global Time (Epoch Time)',
       'Local X (m)', 'Local Y(m)', 'Vehicle Length', 'Vehicle Width',
       'Vehicle Class', 'Vehicle Velocity(km/h)', 'Vehicle Acceleration',
       'Lane Identification', 'Preceding Vehicle', 'Following Vehicle',
       'Spacing Headway', 'Time Headway', 'Bad Object', 'Special Car',
       'Lane Class', 'Vehicle Movement'],
      dtype='object')

In [13]:
folder_name = '02-0_total_volume_speed'

save_TTL_dir = os.path.join(folder_dir, folder_name)

os.makedirs(save_TTL_dir, exist_ok = True) # 해당 경로가 없을 시 폴더 생성, 존재할 경우 건너뛰기

In [14]:
for df, num_LOS in tqdm(zip(df_list, num_LOS_list)):
    
    if len(df) > 0:
    
        total_veh = len(df['Vehicle ID'].unique()) # 총 차량대수
        total_SMS = harmonic(df['Vehicle Velocity(km/h)']) #차량의 공간평균속도(SMS)
    
        tmsec = tmsec_info[(tmsec_info['num'] == num_LOS[0:4]) & (tmsec_info['LOS'] == num_LOS[-1])]['tmsec'].iloc[0] # 해당 대상지, 해당 LOS 총관측시간 추출
    
        # 프레임 당 차량밀도의 전체평균 구하기
        total_pv = pd.pivot_table(df, index = 'Frame ID', values = 'Vehicle ID', aggfunc = {'Vehicle ID': unq}).reset_index()
        veh_by_frame = total_pv['Vehicle ID'].mean() #전체 프레임에 대한 평균이므로 mean을 사용한 것임

        # 전체 평균 교통류율(flowrate, vph) 구하기
        total_flowrate = total_veh/tmsec * 3600 # tmsec은 초 단위이므로, 시(h, 3600s) 단위로 변경해야 하기 때문임
    
        total_df = pd.DataFrame(data = {'Total Vehicle Count(veh)': [total_veh],
                                        'Average SMS(km/h)': [total_SMS],
                                        'Average Flowrate(vph)': [total_flowrate],
                                        'Average Vehicle by Frame(veh/frameID)':[veh_by_frame],
                                        'Average LOS' : num_LOS[-1],
                                        'Total Observation Time(s)': [tmsec]})
    
        save_TTL_file = f'total_volume_SMS_{num_LOS}.csv'
        save_TTL_path = os.path.join(save_TTL_dir, save_TTL_file)
    
        total_df.to_csv(save_TTL_path, encoding = 'cp949')
    
    else:
        pass

4it [00:00,  8.57it/s]


## 차선별 속도 정보

### 차선변경 여부 표시 파생변수 생성 및 저장

In [15]:
df.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1', 'Unnamed: 0.1.1.1',
       'Vehicle ID', 'Frame ID', 'Total Frames', 'Global Time (Epoch Time)',
       'Local X (m)', 'Local Y(m)', 'Vehicle Length', 'Vehicle Width',
       'Vehicle Class', 'Vehicle Velocity(km/h)', 'Vehicle Acceleration',
       'Lane Identification', 'Preceding Vehicle', 'Following Vehicle',
       'Spacing Headway', 'Time Headway', 'Bad Object', 'Special Car',
       'Lane Class', 'Vehicle Movement'],
      dtype='object')

In [16]:
folder_name = '01_density_by_target_LOS_concated_lanechange'

save_Lanechange_dir = os.path.join(folder_dir, folder_name)

os.makedirs(save_Lanechange_dir, exist_ok = True) # 해당 경로가 없을 시 폴더 생성, 존재할 경우 건너뛰기

In [17]:
warnings.filterwarnings(action = 'ignore') # pandas 경고메시지 숨기기

for df, num_LOS in tqdm(zip(df_list, num_LOS_list)):

    df['Lane_record'] = None # 차량의 차로변경기록
    df['Lane_00'] = None # 차량의 첫 차로
    df['Lane_99'] = None # 차량의 마지막 차로
    df['Lane_change'] = None # 차량이 차로변경을 했을 시 'Change' 기록
    
    if len(df) > 0:

        veh_list = df['Vehicle ID'].unique() # 차량 리스트 뽑기
    
        df = df.sort_values(by = 'Frame ID') # 데이터프레임을 Frame ID에 따라서 오름차순 정렬
    
        for veh in veh_list: # 각 차량별로
            Lane_00 = df[df['Vehicle ID'] == veh]['Lane Identification'].iloc[0] # 차량이 처음 진행하던 차로
            Lane_99 = df[df['Vehicle ID'] == veh]['Lane Identification'].iloc[-1] # 차량이 마지막 진행하던 차로
        
            veh_frame_list = df[df['Vehicle ID'] == veh]['Frame ID'].unique() # 해당 차량이 주행하는 동안의 프레임 리스트
        
            Lane_past = Lane_00 # Lane_past(각 차량 현재차로)를 초기화
            Lane_record = Lane_00 # 차로변경기록 초기화
            Lane_present = Lane_00 # 현재차로 초기화
        
            for frm in veh_frame_list : # 각 차량의 각 프레임별로
            
                Lane_past = Lane_present # iteration 돌기 전의 현재차로가 과거차로로 할당
            
                Lane_present = df[(df['Vehicle ID'] == veh) & (df['Frame ID'] == frm)]['Lane Identification'].iloc[0] # 현재 차량이 속한 차로를 할당
            
                if Lane_past == Lane_present: # 현재차로와 직전차로가 같으면 아무 문제 없음
                    pass
            
                else: # 현재차로와 직전 차로가 다를 시
                    idx = df[(df['Vehicle ID'] == veh) & (df['Frame ID'] == frm)]['Lane_change'].index[0] # 차로변경된 행의 인덱스
                    df.at[idx, 'Lane_change'] = 'Change' # 차로변경되자마자 셀에 'Change' 표시
                    Lane_record = str(Lane_record) + '_' + str(Lane_present)
               
            df.loc[df['Vehicle ID'] == veh, 'Lane_00'] = Lane_00 # 차량별 초기 차로번호
            df.loc[df['Vehicle ID'] == veh, 'Lane_99'] = Lane_99 # 차량별 마지막 차로번호
            df.loc[df['Vehicle ID'] == veh, 'Lane_record'] = Lane_record # 차량별 차로변경 기록 입력
            
            save_Lanechange_name = num_LOS + '.csv'
            save_Lanechange_path = os.path.join(save_Lanechange_dir, save_Lanechange_name)
            
            df.to_csv(save_Lanechange_path, encoding = 'cp949')
            
    else:
        pass
    
#warnings.filterwarnings(action = 'default') # pandas 경고메시지 다시 보이기

4it [04:44, 71.07s/it]


### 궤적 데이터 재로드

In [18]:
folder_name = '01_density_by_target_LOS_concated_lanechange'

file_dir = os.path.join(folder_dir, folder_name)

In [19]:
file_list = os.listdir(file_dir)
file_list[0:3]

['01_1_A.csv', '01_2_A.csv', '01_3_A.csv']

In [20]:
df_list = []
num_LOS_list = []

for file in file_list:
    num_LOS = file[0:6]
    
    file_path = os.path.join(file_dir, file)
    
    globals()[f'target{num_LOS}'] = pd.read_csv(file_path, encoding = 'cp949')
    
    df_list.append(globals()[f'target{num_LOS}'])
    num_LOS_list.append(num_LOS)

In [21]:
#target01_1_A.head(3)

### 차선별 교통량, SMS 산정 후 저장
* 차선별 교통량(차량대수)의 산정 기준
    * 시점부 : 끝 지점
    * 종점부1,2는 각 시작지점

In [22]:
folder_name = '02-1_Each_Lane_volume_speed'

save_EachLaneVolume_dir = os.path.join(folder_dir, folder_name)

os.makedirs(save_EachLaneVolume_dir, exist_ok = True) # 해당 경로가 없을 시 폴더 생성, 존재할 경우 건너뛰기

In [23]:
for df, num_LOS in tqdm(zip(df_list, num_LOS_list)):
    
    tmsec = tmsec_info[(tmsec_info['num'] == num_LOS[0:4]) & (tmsec_info['LOS'] == num_LOS[-1])]['tmsec'].iloc[0]
    
    # 시점부, 종점부를 구분하기 : spot == 1이면 시점부, 종점부는 그 외로 간주
    point = num_LOS[3]
    
    if point == 1 : # 시점부이면, 끝시점 차로(Lane_99) 기준 산정
        pv_veh = pd.pivot_table(df,
                                index = ['Lane_99'],
                                values = ['Vehicle ID', 'Vehicle Velocity(km/h)'],
                                aggfunc = {'Vehicle ID': unq, 'Vehicle Velocity(km/h)': harmonic})
        
    else : # 종점부이면, 첫차로(Lane_00) 기준 산정
        pv_veh = pd.pivot_table(df, index = ['Lane_00'],
                                values = ['Vehicle ID', 'Vehicle Velocity(km/h)'],
                                aggfunc = {'Vehicle ID': unq, 'Vehicle Velocity(km/h)': harmonic})
    
    pv_veh.reset_index(inplace = True)
        
    pv_veh['Flow Rate(vph)'] = pv_veh['Vehicle ID'] * 3600/tmsec
    
    save_EachLaneVolume_file = f'EachLaneVolume{num_LOS}.csv'
    save_EachLaneVolume_path = os.path.join(save_EachLaneVolume_dir, save_EachLaneVolume_file)
    
    pv_veh.to_csv(save_EachLaneVolume_path, encoding = 'cp949')

4it [00:00, 38.20it/s]


### 차선변경 차량의 교통량 및 SMS 산정, 저장

In [24]:
folder_name = '02-2_Lanechange_volume_speed'

save_LaneChangeVolume_dir = os.path.join(folder_dir, folder_name)

os.makedirs(save_LaneChangeVolume_dir, exist_ok = True) # 해당 경로가 없을 시 폴더 생성, 존재할 경우 건너뛰기

In [25]:
for df, num_LOS in tqdm(zip(df_list, num_LOS_list)):
    
    tmsec = tmsec_info[(tmsec_info['num'] == num_LOS[0:4]) & (tmsec_info['LOS'] == num_LOS[-1])]['tmsec'].iloc[0]
    
    pv_veh = pd.pivot_table(df, index = ['Lane_record'], values = ['Vehicle ID', 'Vehicle Velocity(km/h)'],
                            aggfunc = {'Vehicle ID': unq, 'Vehicle Velocity(km/h)': harmonic})
    pv_veh.reset_index(inplace = True)
        
    pv_veh['flow_rate(vph)'] = pv_veh['Vehicle ID'] * 3600/tmsec
    
    save_LaneChangeVolume_file = f'LaneChangeVolume{num_LOS}.csv'
    save_LaneChangeVolume_path = os.path.join(save_LaneChangeVolume_dir, save_LaneChangeVolume_file)
    
    pv_veh.to_csv(save_LaneChangeVolume_path, encoding = 'cp949')

4it [00:00, 38.20it/s]


## 과속(overspeed)
> **과속** : 도로 제한속도보다 20km/h 초과 운행한 경우
* 차량의 영상 내 구간 평균속도 산정
    * index는 `Lane Identification, Vehicle ID`
    * value는 `Vehicle Velocity(km/h)`
    * 함수는 그냥 평균
* 과속 여부 판정 파생변수 생성하기 : `overspeed`
* `Lane Identification` 별로 pivot_table 하기 : `overspeed` 만 필터링하여 값의 수를 세기

### 대상지 정보 로드

In [26]:
target_info_dir = 'target_info_A_MH.csv'

In [27]:
target_info = pd.read_csv(target_info_dir, encoding = 'utf-8')

In [28]:
target_info.head()

,place,length,speed_limit,lane_num,order,etc
0,01_1,167,80,4,NaN,NaN
1,01_2,180,80,4,NaN,NaN
2,01_3,150,110,4,NaN,NaN
3,01_4,145,110,4,NaN,NaN


### 필요함수 정의 : overspeed
* 도로 제한속도보다 20km/h 이상 초과운행 시 overspeed를 반환하는 함수

In [29]:
def overspeed(velocity, limit):
    """도로 제한속도(limit)보다 20km/h 이상 초과 운행 시 overspeed 판정 함수"""
    
    if velocity - limit > 20:
        return 'Overspeed'
    else:
        pass

### 과속차량에 'over' 표시하기

In [30]:
ovs_list = []

for df, num_LOS in zip(df_list, num_LOS_list):
    
    df['place'] = num_LOS[0:4] # 01_1
    
    pvt = pd.pivot_table(df,
                         index = ['place', 'Lane Identification', 'Vehicle ID'],
                         values = ['Vehicle Velocity(km/h)', 'Vehicle Acceleration'],
                         aggfunc = {'Vehicle Velocity(km/h)': 'mean', 'Vehicle Acceleration':'mean'})
    pvt.reset_index(inplace = True)
    
    ovs = pd.merge(left = pvt, right = target_info, how = 'left', left_on = 'place', right_on = 'place')
    
    ovs['overspeed'] = ovs[['Vehicle Velocity(km/h)', 'speed_limit']].apply(lambda x: overspeed(*x), axis = 1)
            
    globals()[f'ovs{num_LOS}'] = ovs
    ovs_list.append(ovs)

In [31]:
ovs_list[0].head(3)

,place,Lane Identification,Vehicle ID,Vehicle Acceleration,Vehicle Velocity(km/h),length,speed_limit,lane_num,order,etc,overspeed
0,01_1,U1,10,-0.181515,91.505575,167,80,4,NaN,NaN,None
1,01_1,U1,28,-0.574813,110.759401,167,80,4,NaN,NaN,Overspeed
2,01_1,U1,32,-0.524107,111.084511,167,80,4,NaN,NaN,Overspeed


### 과속차량(over) 필터링 후 처리
* 과속한 차량대수
* 과속한 차량 대수를 교통류율로 환산
* 과속한 차량끼리의 공간평균속도를 구하기

In [32]:
folder_name = '02-3_overspeed'

save_overspeed_dir = os.path.join(folder_dir, folder_name)

os.makedirs(save_overspeed_dir, exist_ok = True) # 해당 경로가 없을 시 폴더 생성, 존재할 경우 건너뛰기

In [33]:
for ovs, num_LOS in zip(ovs_list, num_LOS_list):
    
    ovs_only = ovs[ovs['overspeed'] == 'Overspeed']
    
    ovs_veh = pd.pivot_table(ovs_only, 
                             index = 'Lane Identification',
                             values = ['Vehicle ID', 'Vehicle Velocity(km/h)'],
                             aggfunc = {'Vehicle ID': unq, 'Vehicle Velocity(km/h)': harmonic})
    
    ovs_veh.reset_index(inplace = True)
    
    if len(ovs_veh) > 0:
        print(num_LOS, len(ovs_only))
        #ovs2_veh['flow_rate(vph)'] = ovs2_veh['Vehicle ID_x'] * 3600/tmsec
        save_overspeed_file = f'overspeed{num_LOS}.csv'
        save_overspeed_path = os.path.join(save_overspeed_dir, save_overspeed_file)
        ovs_veh.to_csv(save_overspeed_path, encoding = 'cp949')
        
    else:
        pass

01_1_A 27
01_2_A 40
01_4_A 2


## 급감속(sudden_dec)
* **급감속** : 초당 8km/h 이상 감속 운행하고 속도가 6.0km/h 이상인 경우
    * 화물차의 것을 기준으로 한 정의임
* 이것이야말로 '초'당 피벗 테이블이 필요함

### time_group : 1초 단위로 묶은 파생변수 생성
* 1초 단위 피벗을 하기 위함임

In [34]:
frame_rate = 30

In [35]:
pivot_df_list = []

for df, num_LOS in tqdm(zip(df_list, num_LOS_list)):
    
    df['timespot'] = round(df['Frame ID']/frame_rate) # frame_rate == 30
    
    vehicle_list = df['Vehicle ID'].unique()
    
    pivot_df = pd.pivot_table(df, 
                              index = ['Lane Identification', 'Vehicle ID', 'timespot'],
                              values = ['Vehicle Velocity(km/h)' ,'Vehicle Acceleration'],
                              aggfunc = {'Vehicle Velocity(km/h)': harmonic, 'Vehicle Acceleration' : 'mean'})
    
    pivot_df = pivot_df.reset_index()
    
    pivot_df_list.append(pivot_df)

4it [00:00, 10.17it/s]


In [36]:
pivot_df_list[0]

,Lane Identification,Vehicle ID,timespot,Vehicle Acceleration,Vehicle Velocity(km/h)
0,U1,10,15.0,-0.878970,90.783041
1,U1,10,16.0,0.092216,89.361215
2,U1,10,17.0,0.765685,91.175480
3,U1,10,18.0,0.528754,94.270820
4,U1,10,19.0,-0.810477,92.869941
...,...,...,...,...,...
2056,U4,890,297.0,-2.910350,80.077538
2057,U4,890,298.0,-1.019102,80.502021
2058,U4,890,299.0,-0.380518,78.329688
2059,U4,890,300.0,0.954549,79.124888


### 필요 함수 정의 : sudden_dec
* **급감속** : 초당 8km/h 이상 감속 운행하고 속도가 6.0km/h 이상인 경우
* 가속도를 초속(m/s)으로 통일할 것

In [37]:
def sudden_dec(velocity, acceleration):
    """급감속인 경우 sudden_dec 판정"""
    if velocity >= 6.0 and acceleration <= -8.0 * (1000/3600):
        return 'sudden_dec'
    else:
        pass

### 급감속(sudden_dec) 표시 후 저장

In [38]:
folder_name = '02-4_sudden_dec'

save_SuddenDec_dir = os.path.join(folder_dir, folder_name)

os.makedirs(save_SuddenDec_dir, exist_ok = True) # 해당 경로가 없을 시 폴더 생성, 존재할 경우 건너뛰기

In [39]:
for pivot_df, num_LOS in zip(pivot_df_list, num_LOS_list):
    
    pivot_df['sudden_dec'] = pivot_df[['Vehicle Velocity(km/h)', 'Vehicle Acceleration']].apply(lambda x: sudden_dec(*x), axis = 1)
    
    save_SuddenDec_file = f'SuddenDec{num_LOS}.csv'
    save_SuddenDec_path = os.path.join(save_SuddenDec_dir, save_SuddenDec_file)
    
    pivot_df.to_csv(save_SuddenDec_path, encoding = 'cp949')

In [40]:
pivot_df_list[0].head()

,Lane Identification,Vehicle ID,timespot,Vehicle Acceleration,Vehicle Velocity(km/h),sudden_dec
0,U1,10,15.0,-0.878970,90.783041,None
1,U1,10,16.0,0.092216,89.361215,None
2,U1,10,17.0,0.765685,91.175480,None
3,U1,10,18.0,0.528754,94.270820,None
4,U1,10,19.0,-0.810477,92.869941,None


### 급감속, 급가속 필터링 후 처리
* 급감속/급가속한 차량대수
* 해당 차량 대수를 교통류율로 환산
* 해당 차량끼리의 평균 가속도를 구하기

In [41]:
folder_name = '02-5_sudden_dec_aggregated'

save_SuddenDecAgg_dir = os.path.join(folder_dir, folder_name)

os.makedirs(save_SuddenDecAgg_dir, exist_ok = True) # 해당 경로가 없을 시 폴더 생성, 존재할 경우 건너뛰기

In [42]:
for pivot_df, num_LOS in zip(pivot_df_list, num_LOS_list):
    
    SuddenDecAgg = pd.pivot_table(pivot_df[pivot_df['sudden_dec'] == 'sudden_dec'],
                                  index = ['Lane Identification', 'sudden_dec'],
                                  values = ['Vehicle ID', 'Vehicle Acceleration'],
                                  aggfunc = {'Vehicle ID': unq, 'Vehicle Acceleration': 'mean'})
    
    SuddenDecAgg.reset_index(inplace = True)
    
    if len(SuddenDecAgg) > 0:
        print(num_LOS, len(SuddenDecAgg))
        save_SuddenDecAgg_file = f'SuddenDecAgg{num_LOS}.csv'
        save_SuddenDecAgg_path = os.path.join(save_SuddenDecAgg_dir, save_SuddenDecAgg_file)
        SuddenDecAgg.to_csv(save_SuddenDecAgg_path, encoding = 'cp949')
        
    else:
        pass

01_1_A 4
01_2_A 4
01_3_A 4
01_4_A 4
